# GluonCV Object Detection - AWS Marketplace Model Package

This tutorial walks you through the fundamental usage of GluonCV object detection models in AWS Marketplace.


## Essential Sagemaker imports

In [1]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

##  Create the session

In [2]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
region

'us-west-2'

## Subscribe to the model

Before you can deploy the model, your account needs to be subscribed to it.

- Open the Model Package listing page and search for GluonCV image classification models .
- Read the product overview section and Highlights section to understand the value proposition of our model package.
- View usage information and then additional resources sections. 
- Click on Continue to Subscribe button & then Click on "Accept Offer" button on the next page.
- Next, Continue to Configuration button becomes activated and when you click on the button, you will see that a Product Arn will appear.
- You have successfully subscribed to our model package. You can find it in your SageMaker console. Keep record of the model package arn you have subscribed to.
- Close the page and continue executing this notebook.

In [3]:
modelpackage_arn = 'arn:aws:sagemaker:us-west-2:985964311364:model-package/gluoncv-yolo3-darknet53-coco-1600288930'

## Create Model

In [4]:
from sagemaker import ModelPackage

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='image/jpeg')

# Create a deployable model
model = ModelPackage(role=role,
                     model_package_arn = modelpackage_arn,
                     sagemaker_session = sess,
                     predictor_cls = predict_wrapper)
endpoint_name = 'gluoncv-yolov3'

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


## Create live Inference Endpoint

Now that we have the model created, we can launch the live inference endpoint, this may take a while in the background.

In [5]:
predictor = model.deploy(1, 'ml.m4.xlarge', endpoint_name=endpoint_name)

-----------!

## Let's grab some demo images for inference

In [6]:
!wget https://github.com/zhreshold/gluoncv-sagemaker-examples/raw/master/examples/input/playground.jpg -O test.jpg

--2020-09-16 21:06:37--  https://github.com/zhreshold/gluoncv-sagemaker-examples/raw/master/examples/input/playground.jpg
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zhreshold/gluoncv-sagemaker-examples/master/examples/input/playground.jpg [following]
--2020-09-16 21:06:37--  https://raw.githubusercontent.com/zhreshold/gluoncv-sagemaker-examples/master/examples/input/playground.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195229 (191K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>] 190.65K  --.-KB/s    in 0.03s   

2020-09-16 21:06:37 (7.35 MB/s) - ‘test.jpg’ saved [195229/195229]



We can perform a typical object detection task with the live endpoint by posting the raw image data

In [7]:
file_name = 'test.jpg'
# Open the file and read the image into a bytearray.
with open(file_name, "rb") as image:
  b = bytearray(image.read())

# Perform a prediction
result = predictor.predict(b).decode('utf-8')

with open("output.json", "w") as text_file:
    text_file.write(result)
    
# View the prediction
print(result)

[{"id": "person", "score": 0.9748764038085938, "left": 106, "top": 46, "right": 128, "bottom": 92}, {"id": "person", "score": 0.9437488913536072, "left": 508, "top": 114, "right": 603, "bottom": 272}, {"id": "person", "score": 0.9360765814781189, "left": 164, "top": 80, "right": 200, "bottom": 166}, {"id": "person", "score": 0.9176429510116577, "left": 414, "top": 86, "right": 511, "bottom": 282}, {"id": "person", "score": 0.9088366627693176, "left": 332, "top": 37, "right": 350, "bottom": 80}, {"id": "person", "score": 0.7969743609428406, "left": 366, "top": 58, "right": 385, "bottom": 112}, {"id": "person", "score": 0.7222182154655457, "left": 276, "top": 84, "right": 301, "bottom": 168}, {"id": "person", "score": 0.7201529741287231, "left": 353, "top": 47, "right": 370, "bottom": 96}, {"id": "person", "score": 0.7141261696815491, "left": 383, "top": 39, "right": 399, "bottom": 83}, {"id": "person", "score": 0.7052568793296814, "left": 469, "top": 14, "right": 476, "bottom": 33}, {"i

##  Delete the Endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.

In [27]:
predictor = sage.RealTimePredictor(endpoint_name, sess, content_type='image/jpeg')
predictor.delete_endpoint(delete_endpoint_config=True)